In [1]:
# Get raw data
with open('input/22.txt', 'r') as f:
    rawinput = f.read().strip()

In [8]:
import numpy as np

In [584]:
# Part 1
depth, targ_x, targ_y = [int(j) for i in rawinput.split('\n') for j in i.split(':')[1].split(',')]
# depth, targ_x, targ_y = 510,10,10

x = np.expand_dims(np.arange(targ_x+1), 0)
y = np.expand_dims(np.arange(targ_y+1), 1)
geoindex = np.where((x==0)|(y==0), x*16807 + y*48271, -1)
while np.any(geoindex==-1):
    erosion = np.where(geoindex < 0, geoindex, (geoindex+depth)%20183)
    geoindex[1:, 1:] = np.where(((en:=erosion[:-1,1:])<0) | ((ew:=erosion[1:,:-1])<0), -1, en*ew)
geoindex[targ_y, targ_x] = 0    
erosion[targ_y, targ_x] = erosion[0,0]

np.sum(np.mod(erosion,3)).item()

11575

In [ ]:
# Part 2
class Spelunk(object):
    def __init__(self, depth, target_loc):
        self.depth = depth
        self.target_loc = target_loc
        self.geoindex = np.zeros((1,1), dtype=int)
        self.erosion = np.mod(self.geoindex+self.depth, 20183)
        self.reg_type = np.mod(self.erosion, 3)
        self.min_time = -np.ones((1,1,3), dtype=int)
        
    def expand_e(self):
        addcol = [*zip(*[[(y:=0 if (i,self.geoindex.shape[1]) == self.target_loc
                              else j*z if i 
                              else self.geoindex[0,-1]+16807), 
                          (z:=(y+self.depth)%20183),
                          np.mod(z,3)]
                         for i,j in enumerate(self.erosion[:,-1])])]
        (self.geoindex, 
         self.erosion,
         self.reg_type) = [np.column_stack((j, addcol[i]))
                           for i,j in enumerate([self.geoindex, 
                                                 self.erosion,
                                                 self.reg_type])]
        self.min_time = np.pad(self.min_time, ((0,0),(0,1),(0,0)), constant_values=-1)
        self.min_time[0,0,1] = 0
        
    def expand_s(self):
        addrow = [*zip(*[[(y:=0 if (self.geoindex.shape[0],i) == self.target_loc
                              else j*z if i
                              else self.geoindex[-1,0]+48271), 
                          (z:=(y+self.depth)%20183),
                          np.mod(z,3)]
                         for i,j in enumerate(self.erosion[-1])])]
        (self.geoindex, 
         self.erosion,
         self.reg_type) = [np.vstack((j, addrow[i]))
                           for i,j in enumerate([self.geoindex, 
                                                 self.erosion,
                                                 self.reg_type])]
        self.min_time = np.pad(self.min_time, ((0,1),(0,0),(0,0)), constant_values=-1)
        
    def find_path(self):
        loc = np.array([[0,0]])
        kit = np.array([1])
        time = np.array([0])
        mvhist = np.empty((1,0), dtype='<U1')
        pcd = 100
        while pcd:
            if np.any(loc[:,0]+1 == self.reg_type.shape[0]):
                self.expand_s()
            if np.any(loc[:,1]+1 == self.reg_type.shape[1]):
                self.expand_e()
        
            loc = np.repeat(loc, 6, axis=0) + np.tile([[-1,0], [1,0], [0,-1], [0,1], [0,0], [0,0]], (loc.shape[0],1))
            kit = np.mod(np.repeat(kit, 6, axis=0) + np.tile([0,0,0,0,1,2], kit.shape[0]), 3)
            time = np.repeat(time, 6, axis=0) + np.tile([1,1,1,1,7,7], time.shape[0])
            mvhist = np.column_stack((np.repeat(mvhist, 6, axis=0), np.tile(list('NSWE12'), mvhist.shape[0])))
        
            filt = (np.all(loc >= 0, axis=1)
                    & (self.reg_type[loc[:,0], loc[:,1]] != kit)
                    & (((z:=self.min_time[loc[:,0], loc[:,1], kit]) < 0) | (time < z)))
            loc, kit, time, mvhist = [i[filt] for i in [loc, kit, time, mvhist]]
        
            filt = np.any((z:=(y:=np.column_stack((loc, kit, time)))
                            [(sortix:=np.lexsort([y[:,i] for i in range(y.shape[1]-1,-1,-1)]))])
                        != np.append(-np.ones_like(z[:1]), z[:-1], axis=0), 
                        axis=1)[np.argsort(sortix)]
            loc, kit, time, mvhist = [i[filt] for i in [loc, kit, time, mvhist]]
        
            self.min_time[loc[:,0], loc[:,1], kit] = time
            
            if (np.all(np.subtract(self.min_time.shape[:2], self.target_loc) > 0) 
                and (z:=self.min_time[*self.target_loc, 1]) > 0):
                pcd -= 1
        return z.item()

self = Spelunk(depth, (targ_y,targ_x))
self.find_path()

KeyboardInterrupt: 

In [587]:
683*678

463074

In [589]:
114*878

100092